In [51]:
%load_ext autoreload
%autoreload 2
import sys
import gc

from multiprocessing import cpu_count, Pool
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data as data_utils

from tqdm import tqdm

sys.path.append('../')
from wiki.utils import clock
from wiki import rnn, rnn_predict, newphet, val
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
torch.manual_seed(1)
torch.cuda.set_device(0)

In [53]:
batch_size = 1024
pred_len = 62

In [54]:
base_dir = '../data/'
train_df = pd.read_csv(base_dir+'train_2.csv').fillna(0)

In [55]:
X = train_df.drop('Page', axis=1).values
X, scaler = rnn.scale_values(X)

In [56]:
X[:,-2*pred_len-430:-2*pred_len,:].shape

(145063, 430, 1)

In [57]:
trainloader = data_utils.DataLoader(
    data_utils.TensorDataset(
        torch.from_numpy(X[:,-2*pred_len-430:-2*pred_len,:]).float(),
        torch.from_numpy(X[:,-2*pred_len:-pred_len,:]).float()
    ),
    batch_size=batch_size, shuffle=False
)

In [58]:
valloader = data_utils.DataLoader(
    data_utils.TensorDataset(
        torch.from_numpy(X[:,:-pred_len,:]).float(),
        torch.from_numpy(X[:,-pred_len:,:]).float()
    ),
    batch_size=batch_size, shuffle=False
)

In [59]:
loss_func = nn.L1Loss()
model = rnn.RNN(loss_func=loss_func).cuda()

In [60]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
save_best_path = base_dir+'rnn_stage2_v2_lr1.mdl'
with clock():
    model.fit(trainloader, valloader, optimizer=optimizer, num_epochs=20, save_best_path=save_best_path)


EPOCH 1
Elapsed time 3.493138074874878 seconds
Elapsed time 3.4933879375457764 seconds


RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1503970438496/work/torch/lib/THC/generic/THCStorage.cu:66

In [ ]:
save_best_path = base_dir+'rnn_stage2_v2_lr1.mdl'
model.load_state_dict(torch.load(save_best_path))

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
save_best_path = base_dir+'rnn_stage2_v2_lr2.mdl'
with clock():
    model.fit(trainloader, valloader, optimizer=optimizer, num_epochs=10, save_best_path=save_best_path)

In [ ]:
model = rnn.RNN().cuda()
save_best_path = base_dir+'rnn_stage2_v2_lr2.mdl'
model.load_state_dict(torch.load(save_best_path))

In [ ]:
outputs, targets, sequences = model.predict(valloader)

In [ ]:
_, predictions = rnn_predict.combine_prediction_data(outputs, targets, sequences)

In [ ]:
base_dir = '../data/'
train_df = pd.read_csv(base_dir+'train_2.csv')
X = train_df.drop('Page', axis=1).values

In [ ]:
predictions = scaler.inverse_transform(predictions.T).T
true = X

In [ ]:
smapes = val.smape(true[:,-60:], predictions[:,-60:], axis=1)
smapes_clipped = val.smape(true[:,-60:], predictions[:,-60:].round().clip(0), axis=1)

In [ ]:
np.nanmean(smapes), np.nanmean(smapes_clipped)

In [ ]:
smapes_clipped = val.smape(true[:,-60:], predictions[:,-60:].round().clip(0), axis=0)

In [ ]:
smapes_sequences = val.smape(true[:,-60:], predictions[:,-60:].round().clip(0), axis=1)

In [ ]:
smapes_clipped.shape

In [ ]:
moving = []
for i in range(smapes_clipped.shape[0]):
    moving.append(np.nanmean(smapes_clipped[:i+1]))
moving = np.array(moving)

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
plt.figure(figsize=(10,10))
plt.plot(moving)

In [ ]:
median = np.nanmedian(true[:,-49-60:-60], axis=1).reshape(-1,1)

In [ ]:
median.shape

In [ ]:
smapes_median = val.smape(true[:,-60:], median, axis=0)

In [ ]:
smapes_median_sequences = val.smape(true[:,-60:], median, axis=1)

In [ ]:
m_moving = []
for i in range(smapes_median.shape[0]):
    m_moving.append(np.nanmean(smapes_median[:i+1]))
m_moving = np.array(m_moving)

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(m_moving)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.grid(True)
ax.plot(moving-m_moving)
plt.show()

In [ ]:
joined = np.amin(np.stack([smapes_median_sequences, smapes_sequences]), axis=0) ; joined.shape

In [ ]:
np.nanmean(joined)

In [ ]:
np.nanmean(smapes_median_sequences)

In [ ]:
predictions.shape

In [61]:
median.shape

(145063, 1)

In [73]:
median_broadcast = np.broadcast_to(median, (median.shape[0],20))

In [74]:
median_broadcast.shape

(145063, 20)

In [75]:
altered = np.concatenate([predictions[:, -60:-20], median_broadcast], axis=1)

In [76]:
altered.shape

(145063, 60)

In [77]:
smapes_mixture = val.smape(true[:,-60:], altered.clip(0).round(), axis=1)

../wiki/val.py:6: RuntimeWarning: invalid value encountered in true_divide
  diff = np.abs(y_true - y_pred) / denominator
../wiki/val.py:8: RuntimeWarning: Mean of empty slice
  return np.nanmean(diff) if axis is None else np.nanmean(diff, axis=axis)


In [78]:
np.nanmean(smapes_mixture)

41.7182098667841

## 